<a href="https://colab.research.google.com/github/DominicJW/CogRobCW/blob/main/SubmissionCogRob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torch import nn,optim
from torch.nn import functional as F
from tqdm import tqdm
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import sys
from torch.utils.data import DataLoader
from google.colab import drive
import os
from torch.cuda.amp import autocast, GradScaler #For mixed precision, faster gpu execution (didn't speed up much)
import time


# drive.mount('/content/drive')
# drive_path = '/content/drive/My Drive/'

#If not using drive
drive_path = "./"

!wget -O tmpfile https://www.dropbox.com/scl/fo/4adbsoqv3wbhmwyofuno9/AFNRg8Ps8O21x2a1hJ9WSAk?rlkey=1t9kvu7slm49b17z7fkgfjgax&st=49tsyyw4&dl=1
!unzip ./tmpfile
!mkdir CogRob
!mv bs_lr_wd_test_extended_bs_lr_range.csv bs_lr_wd_test.csv df256vs64.csv joined_df.csv ./CogRob/



!pip install onnx


--2024-04-22 08:54:36--  https://www.dropbox.com/scl/fo/4adbsoqv3wbhmwyofuno9/AFNRg8Ps8O21x2a1hJ9WSAk?rlkey=1t9kvu7slm49b17z7fkgfjgax
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc636d0ed95aa66e85f9bbea74f4.dl.dropboxusercontent.com/zip_download_get/B07sc-EPzyDdfIsADlVnaTnJkD6s944ii-2ed3etpOJhhf5Gbp43aVcNId1ia8PksfSFNKdPwvEe2v7eT_0WXuSmEoT8qrta7Ct_w-D0KNDvOw# [following]
--2024-04-22 08:54:37--  https://uc636d0ed95aa66e85f9bbea74f4.dl.dropboxusercontent.com/zip_download_get/B07sc-EPzyDdfIsADlVnaTnJkD6s944ii-2ed3etpOJhhf5Gbp43aVcNId1ia8PksfSFNKdPwvEe2v7eT_0WXuSmEoT8qrta7Ct_w-D0KNDvOw
Resolving uc636d0ed95aa66e85f9bbea74f4.dl.dropboxusercontent.com (uc636d0ed95aa66e85f9bbea74f4.dl.dropboxusercontent.com)... 162.125.6.15, 2620:100:6017:15::a27d:20f
Connecting to uc636d0ed95aa66e85f9bbea74f4.dl.dropbo

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.bn2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.bn3 = nn.BatchNorm1d(120)
        self.fc2 = nn.Linear(120, 84)
        self.bn4 = nn.BatchNorm1d(84)
        self.fc3 = nn.Linear(84, 10)

    @autocast()
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.bn3(self.fc1(x)))
        x = F.relu(self.bn4(self.fc2(x)))
        x = self.fc3(x)
        return x


class BlankScheduler:
    @classmethod
    def step():
        return

/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


In [ ]:
dummy_input = torch.randn(64, 3, 32, 32)


output_onnx_file = 'lenet1.onnx'

torch.onnx.export(LeNet(),
                  dummy_input,
                  output_onnx_file,
                  input_names=['input'],
                  output_names=['output'])

print(f'Model has been exported to {output_onnx_file}')

Model has been exported to lenet1.onnx


In [ ]:
###Isn't actually needed if just using the csv files from my dropbox

transform = transforms.Compose(
    [
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]

    )

batch_size = 256 #Different batch sizes are used

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)



train_size = int(0.8 * len(train_set))  # 80% for training
val_size = len(train_set) - train_size  # 20% for validation
train_set, val_set = torch.utils.data.random_split(train_set, [train_size, val_size])


train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True,pin_memory=True,num_workers= 2)

val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                          shuffle=False)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                         shuffle=False)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def search_batch_size_lr(optimizer_class,bs_range=(7,10),lr_range = (-5,-2),wd_range=(-5,-1),lr_cnt = 4,wd_cnt=4,epochs=20,fname=f"{time.time()}",ModelClass = LeNet):
    df = pd.DataFrame(columns=["epoch","batch_size","train_loss","val_loss","val_acc","WD","lr"])
    wd_trials = [10**i for i in range(wd_range[0],wd_range[1]+1,1)] + [0]
    lr_trials = np.logspace(start=lr_range[0], stop=lr_range[1], num=lr_cnt)
    bs_trials = [2**i for i in range(bs_range[1],bs_range[0]-1,-1)]

    print(f"Batch size range {bs_trials}")
    print(f"learn rate range {lr_trials}")
    print(f"weight decays  {wd_trials}")
    print(f"Model Class {ModelClass}")
    for bs in bs_trials:
        train_loader = DataLoader(train_set,batch_size=bs)
        val_loader = DataLoader(val_set,batch_size=bs)
        for lr in lr_trials:
            for wd in wd_trials:
                model = ModelClass()
                model.to("cuda")
                optimizer = optimizer_class(model.parameters(),lr=lr,weight_decay=wd)
                results = train(model,epochs,optimizer,BlankScheduler,train_loader,val_loader)
                results["lr"] = lr
                results["batch_size"] = bs
                results["WD"] = wd
                df = pd.concat((results,df))
                df.to_csv(os.path.join(drive_path,f"CogRob/{fname}.csv"))
    return df

In [ ]:
# df = search_batch_size_lr(optim.AdamW,bs_range=(7,10),lr_range = (-5,-2),wd_range=(-5,-1),lr_cnt = 4,wd_cnt=4,epochs=5,fname=f"{time.time()}") #4*5*5*4*time_per_epoch = 400 * time_per_epoch = 400 * 15s = 100mins
# df.to_csv(os.path.join(drive_path,f"CogRob/bs_lr_wd_test.csv"))

df = pd.read_csv(os.path.join(drive_path,"CogRob/bs_lr_wd_test.csv"))

In [ ]:
df_at_ep_5 = df.reset_index(drop = True)
df_at_ep_5 = df_at_ep_5[df_at_ep_5["epoch"] ==5 ]

idx = df_at_ep_5.groupby(['lr', 'batch_size'])['val_acc'].idxmax()
result_df = df_at_ep_5.loc[idx].reset_index(drop=True)



fig = go.Figure(data=go.Heatmap(
        z=result_df["val_acc"],
        x=result_df["batch_size"].astype("string"),
        y=result_df["lr"].astype("string"),
        colorscale='gray',
        zmin = 0,
        zmax = 1
    ))

fig.update_layout(
    title='Learn rate X Batch Size Validation Accuracy Heatmap',
    xaxis_title='Batch size',
    yaxis_title='Learn rate',
)


fig.update_traces(
    colorbar=dict(
        title='Validation Accuracy',
        titleside='right',
    )
)

fig.show()

df_best_lr = df_at_ep_5[df_at_ep_5["lr"] == 0.01]

fig = go.Figure(data=go.Heatmap(
        z=df_best_lr["val_acc"],
        x=df_best_lr["batch_size"].astype("string"),
        y=df_best_lr["WD"].astype("string"),
        colorscale='gray',
        zmin = 0,
        zmax = 1
    ))

fig.update_layout(
    title='Weight Decay x Batch Size Accuracy Heatmap at Learn rate 0.01 ',
    xaxis_title='Batch size',
    yaxis_title='Weight Decay'
)

fig.show()

df_next_best_lr = df_at_ep_5[df_at_ep_5["lr"] == 0.001]


fig = go.Figure(data=go.Heatmap(
        z=df_next_best_lr["val_acc"],
        x=df_next_best_lr["batch_size"].astype("string"),
        y=df_next_best_lr["WD"].astype("string"),
        colorscale='gray',


    ))

fig.update_layout(
    title='Weight Decay Batch Size Heatmap',
    xaxis_title='Batch size',
    yaxis_title='Weight Decay'
)

fig.show()

Above: initial graphs drawn to inform any further examiniations

In [ ]:
# new_df = search_batch_size_lr(optim.AdamW,bs_range=(6,7),lr_range = (-3,-1),wd_range=(-5,-2),lr_cnt = 3,wd_cnt=3,epochs=5,fname=f"{time.time()}") #2*3*4*5 * 15s = 30mins
# new_df.to_csv(os.path.join(drive_path,f"CogRob/bs_lr_wd_test_extended_bs_lr_range.csv"))
# new_df = pd.read_csv(os.path.join(drive_path,f"CogRob/bs_lr_wd_test_extended_bs_lr_range.csv"))  ##If already loaded
# df = pd.read_csv(os.path.join(drive_path,"CogRob/bs_lr_wd_test.csv"))
# df = df.drop(df.columns[0],axis = 1)
# df = df.reset_index(drop = True)
# new_df = new_df.reset_index(drop = True)
# joined_df = pd.concat((df,new_df),ignore_index=True,axis = 0)
# joined_df.to_csv(os.path.join(drive_path,f"CogRob/joined_df.csv"))
#Skip above is already have joined_df.csv

joined_df = pd.read_csv((os.path.join(drive_path,f"CogRob/joined_df.csv")))

In [ ]:
joined_5 = joined_df[joined_df["epoch"] == 5]
idx = joined_5.groupby(['lr', 'batch_size'])['val_acc'].idxmax()
result_df = joined_5.loc[idx].reset_index(drop=True)

fig = go.Figure(data=go.Heatmap(
        z=result_df["val_acc"],
        x=result_df["batch_size"].astype("string"),
        y=result_df["lr"].astype("string"),
        colorscale='viridis',
        zmin = 0.2,
        zmax = 0.65

    ))

fig.update_layout(
    title='Learn Rate X Batch Size Validation Accuracy Heatmap at Epoch 5 (taken best from variable weight decay)',
    xaxis_title='Batch Size',
    yaxis_title='Learn Rate',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=24),
    yaxis_title_font=dict(size=24),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),



)

fig.update_traces(
    colorbar=dict(
        title='Validation Accuracy',
        titleside='right',
        titlefont=dict(size=24),
        tickfont=dict(size=20)
    )
)

fig.update_xaxes(categoryorder = "array",categoryarray=["64","128","256","512","1024"])
fig.show()


In [ ]:
df_best_lr = joined_5[joined_5["lr"] == 0.01]

fig = go.Figure(data=go.Heatmap(
        z=df_best_lr["val_acc"],
        x=df_best_lr["batch_size"].astype("string"),
        y=df_best_lr["WD"].astype("string"),
        colorscale='cividis',
        zmin = 0.50,
        zmax = 0.65
    ))

fig.update_layout(
    title='Weight Decay x Batch Size Validation Accuracy Heatmap at Epoch 5 Learn Rate 0.01',
    xaxis_title='Batch size',
    yaxis_title='Weight Decay',
        title_font=dict(size=24),
    xaxis_title_font=dict(size=24),
    yaxis_title_font=dict(size=24),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),



)




fig.update_traces(
    colorbar=dict(
        title='Validation Accuracy',
        titleside='right',
        titlefont=dict(size=24),
        tickfont=dict(size=20)
    )
)


fig.update_xaxes(categoryorder = "array",categoryarray=["64","128","256","512","1024"])
fig.update_yaxes(categoryorder = "array",categoryarray=['0.0','1e-05', '0.0001', '0.001', '0.01'])

fig.show()

In [ ]:
df_next_best_lr = joined_5[joined_5["lr"] == 0.001]


fig = go.Figure(data=go.Heatmap(
        z=df_next_best_lr["val_acc"],
        x=df_next_best_lr["batch_size"].astype("string"),
        y=df_next_best_lr["WD"].astype("string"),
        colorscale='cividis',
        zmin = 0.50,
        zmax = 0.65

    ))

fig.update_layout(
    title='Weight Decay x Batch Size Validation Accuracy Heatmap at Epoch 5 Learn Rate 0.001',
    xaxis_title='Batch size',
    yaxis_title='Weight Decay',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=24),
    yaxis_title_font=dict(size=24),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),



)




fig.update_traces(
    colorbar=dict(
        title='Validation Accuracy',
        titleside='right',
        titlefont=dict(size=24),
        tickfont=dict(size=20)
    )
)


fig.update_xaxes(categoryorder = "array",categoryarray=["64","128","256","512","1024"])
fig.update_yaxes(categoryorder = "array",categoryarray=['0.0','1e-05', '0.0001', '0.001', '0.01'])

fig.show()

In [ ]:
def search_batch_size_lr_2_bs(optimizer_class,bs_comp=(7,10),lr_range = (-5,-2),wd_range=(-5,-1),lr_cnt = 4,wd_cnt=4,epochs=20,fname=f"{time.time()}",ModelClass = LeNet):
    df = pd.DataFrame(columns=["epoch","batch_size","train_loss","val_loss","val_acc","WD","lr"])
    wd_trials = [10**i for i in range(wd_range[0],wd_range[1]+1,1)] + [0]
    lr_trials = np.logspace(start=lr_range[0], stop=lr_range[1], num=lr_cnt)
    bs_trials = [2**i for i in bs_comp]
    print(f"Batch size range {bs_trials}")
    print(f"learn rate range {lr_trials}")
    print(f"weight decays  {wd_trials}")
    print(f"Model Class {ModelClass}")
    for bs in bs_trials:
        train_loader = DataLoader(train_set,batch_size=bs)
        val_loader = DataLoader(val_set,batch_size=bs)
        for lr in lr_trials:
            for wd in wd_trials:
                model = ModelClass()
                model.to("cuda")
                optimizer = optimizer_class(model.parameters(),lr=lr,weight_decay=wd)
                results = train(model,epochs,optimizer,BlankScheduler,train_loader,val_loader)
                results["lr"] = lr
                results["batch_size"] = bs
                results["WD"] = wd
                df = pd.concat((results,df))
                df.to_csv(os.path.join(drive_path,f"CogRob/{fname}.csv"))
    return df

# df_256_vs_64 = search_batch_size_lr_2_bs(optim.AdamW,bs_comp=(6,8),lr_range = (-3,-2),wd_range=(-5,-1),lr_cnt = 2,wd_cnt=4,epochs=20,fname=f"{time.time()}") # 2*2*5*20*15 = 100mins (lower bound) : reality 150mins
# df_256_vs_64.to_csv(os.path.join(drive_path,f"CogRob/df256vs64.csv"))
df_256_vs_64 = pd.read_csv(os.path.join(drive_path,f"CogRob/df256vs64.csv"))
df_256_vs_64 = df_256_vs_64.drop(df_256_vs_64.columns[0],axis = 1)


Firstly use opportunity to compare the results at epoch 5

In [ ]:
df256_vs_64_at5 = df_256_vs_64[df_256_vs_64["epoch"] == 5]

idx = df256_vs_64_at5.groupby(['lr', 'batch_size'])['val_acc'].idxmax()
result_df = df256_vs_64_at5.loc[idx].reset_index(drop=True)

fig = go.Figure(data=go.Heatmap(
        z=result_df["val_acc"],  # Z values
        x=result_df["batch_size"].astype("string"),  # X values
        y=result_df["lr"].astype("string"),  # Y values
        colorscale='viridis',  # Color scale
        zmin = 0.2,
        zmax = 0.65
    ))

fig.update_layout(
    title='Learn rate X Batch Size Validation Accuracy Heatmap At Epoch 5',
    xaxis_title='Batch size',
    yaxis_title='Learn rate',
title_font=dict(size=24),
    xaxis_title_font=dict(size=24),
    yaxis_title_font=dict(size=24),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),



)




fig.update_traces(
    colorbar=dict(
        title='Validation Accuracy',
        titleside='right',
        titlefont=dict(size=24),
        tickfont=dict(size=20)
    )
)

fig.update_xaxes(categoryorder = "array",categoryarray=["64","256"])
fig.show()


Similar distribusion and values as the equivelent section of the corresponding  chart above

In [ ]:

df_best_lr = df256_vs_64_at5[df256_vs_64_at5["lr"] == 0.01]

# Using plotly to create the heatmap
fig = go.Figure(data=go.Heatmap(
        z=df_best_lr["val_acc"],  # Z values
        x=df_best_lr["batch_size"].astype("string"),  # X values
        y=df_best_lr["WD"].astype("string"),  # Y values
        colorscale='cividis',  # Color scale
        zmin = 0.50,
        zmax = 0.65
    ))

fig.update_layout(
    title='Weight Decay Batch Size Heatmap At Epoch 5 for Learn Rate 0.01 (Second Trial)',
    xaxis_title='Batch size',
    yaxis_title='Weight Decay',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=24),
    yaxis_title_font=dict(size=24),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),



)




fig.update_traces(
    colorbar=dict(
        title='Validation Accuracy',
        titleside='right',
        titlefont=dict(size=24),
        tickfont=dict(size=20)
    )
)

fig.update_xaxes(categoryorder = "array",categoryarray=["64","256"])
fig.update_yaxes(categoryorder = "array",categoryarray=['0.0','1e-05', '0.0001', '0.001', '0.01'])

fig.show()

In [ ]:
df_next_best_lr = df256_vs_64_at5[df256_vs_64_at5["lr"] == 0.001]

# Using plotly to create the heatmap
fig = go.Figure(data=go.Heatmap(
        z=df_next_best_lr["val_acc"],  # Z values
        x=df_next_best_lr["batch_size"].astype("string"),  # X values
        y=df_next_best_lr["WD"].astype("string"),  # Y values
        colorscale='cividis',  # Color scale
        zmin = 0.50,
        zmax = 0.65
    ))

fig.update_layout(
    title='Weight Decay Batch Size Heatmap At Epoch 5 for Learn Rate 0.001 (Second Trial)',
    xaxis_title='Batch size',
    yaxis_title='Weight Decay',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=24),
    yaxis_title_font=dict(size=24),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),

    width=800,  # Width in pixels
    height=600  # Height in pixels



)




fig.update_traces(
    colorbar=dict(
        title='Validation Accuracy',
        titleside='right',
        titlefont=dict(size=24),
        tickfont=dict(size=20)
    )
)

fig.update_xaxes(categoryorder = "array",categoryarray=["64","256"])
fig.update_yaxes(categoryorder = "array",categoryarray=['0.0','1e-05', '0.0001', '0.001', '0.01'])

fig.show()

The overall pattern exists for learn rate, The specific values are different. This shows some variability, likely due to random initialisiation and shuffleing of the data. This behavior is known to be less pronounced in larger models.
The effect is more pronounced for weight decay as we see an entirely different distribution of accuracies accross weight decays. This calls into question the validity of the results. But the code here, not accounting for failed runs, and intiial exploration takes about four hours to run in total. We cannot afford to run multiple repeats of these tests to a number where statistical tests can be used, which is around 30. In total that would be 120 hours.
In reterospect, a better approach would have been to narrow the hyper parameter search, use a big (more deterministic) model, and to have initially ran multiple trials of each valuation for the hyperparamteters.

Below are some initial looks at batch size x learn rate validation accuracy across a few epochs. the lower end of this achieved better accuracy.

In [ ]:
df256_vs_64_at5 = df_256_vs_64[df_256_vs_64["epoch"] == 10]
idx = df256_vs_64_at5.groupby(['lr', 'batch_size'])['val_acc'].idxmax()
result_df = df256_vs_64_at5.loc[idx].reset_index(drop=True)

fig = go.Figure(data=go.Heatmap(
        z=result_df["val_acc"],  # Z values
        x=result_df["batch_size"].astype("string"),  # X values
        y=result_df["lr"].astype("string"),  # Y values
        colorscale='viridis',  # Color scale
        zmin = 0.2,
        zmax = 0.65
    ))

fig.update_layout(
    title='Learn Rate X Batch Size Validation Accuracy Heatmap At Epoch 10',
    xaxis_title='Batch Size',
    yaxis_title='Learn Rate',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=24),
    yaxis_title_font=dict(size=24),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),



)




fig.update_traces(
    colorbar=dict(
        title='Validation Accuracy',
        titleside='right',
        titlefont=dict(size=24),
        tickfont=dict(size=20)
    )
)

fig.update_xaxes(categoryorder = "array",categoryarray=["64","256"])
fig.show()


In [ ]:
df256_vs_64_at5 = df_256_vs_64[df_256_vs_64["epoch"] == 15]
idx = df256_vs_64_at5.groupby(['lr', 'batch_size'])['val_acc'].idxmax()
result_df = df256_vs_64_at5.loc[idx].reset_index(drop=True)

fig = go.Figure(data=go.Heatmap(
        z=result_df["val_acc"],  # Z values
        x=result_df["batch_size"].astype("string"),  # X values
        y=result_df["lr"].astype("string"),  # Y values
        colorscale='viridis',  # Color scale
        zmin = 0.2,
        zmax = 0.65
    ))

fig.update_layout(
    title='Learn Rate X Batch Size Validation Accuracy Heatmap At Epoch 15',
    xaxis_title='Batch size',
    yaxis_title='Learn Rate',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=24),
    yaxis_title_font=dict(size=24),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),



)




fig.update_traces(
    colorbar=dict(
        title='Validation Accuracy',
        titleside='right',
        titlefont=dict(size=24),
        tickfont=dict(size=20)
    )
)

fig.update_xaxes(categoryorder = "array",categoryarray=["64","256"])
fig.show()

In [ ]:
df256_vs_64_at5 = df_256_vs_64[df_256_vs_64["epoch"] == 20]
idx = df256_vs_64_at5.groupby(['lr', 'batch_size'])['val_acc'].idxmax()
result_df = df256_vs_64_at5.loc[idx].reset_index(drop=True)

fig = go.Figure(data=go.Heatmap(
        z=result_df["val_acc"],  # Z values
        x=result_df["batch_size"].astype("string"),  # X values
        y=result_df["lr"].astype("string"),  # Y values
        colorscale='viridis',  # Color scale
        zmin = 0.2,
        zmax = 0.65
    ))

fig.update_layout(
    title='Learn Rate X Batch Size Validation Accuracy Heatmap At Epoch 20',
    xaxis_title='Batch size',
    yaxis_title='Learn rate',title_font=dict(size=24),
    xaxis_title_font=dict(size=24),
    yaxis_title_font=dict(size=24),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),



)




fig.update_traces(
    colorbar=dict(
        title='Validation Accuracy',
        titleside='right',
        titlefont=dict(size=24),
        tickfont=dict(size=20)
    )
)

fig.update_xaxes(categoryorder = "array",categoryarray=["64","256"])
fig.show()

Interestingly, higher number of epochs (10,15,20) had lower maximum validation accuracies.

To investigate this I plot validation accuracy against epoch, for the two different learn rates, for batch size 64. Taking the best validation accuracy given (meaning picking whichever weight decay resulted in the highest accuracy.)
I then choose the datapoint with the highst accuracy as the optimal setting for hyperparameters. I then plot a graph of accuracy vs epoch for that weight decay and learn rate. A final plot of training loss vs validation loss across the epochs for this optimal value will also help to answer the question of if   overfitting is causing the (small) reduction in validation accuracy at higher epochs.

In [ ]:
df_64 = df_256_vs_64[df_256_vs_64["batch_size"] == 64]

In [ ]:
idx = df_64.groupby(['epoch', 'lr'])['val_acc'].idxmax()
result_df = df_64.loc[idx].reset_index(drop=True)


In [ ]:
fig = go.Figure(data=px.line(result_df,x= "epoch",y="val_acc",color = "lr",hover_data = {"WD":True}))

fig.update_layout(
    title='Top Validation Accuracy Across Epochs For Learn Rates 0.001, 0.01',
    xaxis_title='Epoch',
    yaxis_title='Validation Accuracy',title_font=dict(size=24),
    xaxis_title_font=dict(size=24),
    yaxis_title_font=dict(size=24),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),
    legend_title_text='Learn Rate',
    legend_title_font=dict(size=24),
    legend_font=dict(size=20)



)


fig.show()

Both lines of this graph are quite unstable. This may be becayse different values for WD are used, so it is not showing a real training trajectory. To have a comparison, I will plotgraphs with WD = 0.0, 0.01, 0.0001 so I can make a comparison. I will use colour to denote weight decay and line dash to denote learning rate

In [ ]:

result_df = df_64[(df_64["WD"] == 0) | (df_64["WD"] == 0.01)]
fig = go.Figure(data=px.line(result_df,x= "epoch",y="val_acc",color = "WD",hover_data = {"WD":True,"lr":True},line_dash="lr"))
fig.update_layout(
    title='Validation Accuracy Across Epochs For Learn Rates 0.001, 0.01 x Weight Decays 0.0, 0.01',
    xaxis_title='Epoch',
    yaxis_title='Validation Accuracy',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=24),
    yaxis_title_font=dict(size=24),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),
    legend_title_text='Weight Decay, Learn Rate',
    legend_title_font=dict(size=24),
    legend_font=dict(size=20)
)


fig.show()

Although for the close together values we cannot conclusively say one is better than the other due to statistical fluctuations, we can see that the optimal weight decay for each of two learn rates is different, we can see this because the blue dashed line is above the red dashed line. quite consistently and often with a large difference for the epochs until epoch 13. nevertheless the optimal epochs were earler for both (signifying that at learn rate 0.001 weight decay 0.0 is better than 0.01).
For the solid lines (learn rate 0.01) we see the blue line is more eratic than the red line in earlier training, though reaching the best validation accuracy of this learn rate at epoch 7 of 0.6294, compared to the best validation accuracy of the solid blue line at epoch 4 of 0.6255. Since these values are so close together, and an earlier comparison of two runs of the same test yielded different results, we cannot draw any conclusions from the difference between these two values.
What we can see is that the solid blue line s the most eratic, though reaching a higher accuracy. The eraticness shows instability, and unpredictiablity and may hint at unreliability of that hyperparameter setting.
The lower learn rate for both settings of WD show more stability than the higher ones. The lower learn rate is infact the recommended default setting of the Adam optimiser. We also see that in general the dahsed blue line is above the dashed red line quite consistently. This suggests that a weight decay of zero is better than a weight decay of 0.01 for this setting.
This is likely due to the modest size of LeNet for the complexity of this task, meaning that the model struggles to fit the training data, meaning it sufferes from the over regularisation, which prevents it from fitting the training data enough. Its low power for this task means that it is unlikely to overfit.
To check this I will draw validation loss against training loss throughout the training, for the dashed blue line (lr 0.001, WD 0) if any of the lines were to overfit, it would most likely be this one. And indeed in the graph above, at higher epochs its valdation accuracy is overtaken by the dashed red line, hinting the dashed redline can handle more iterations without overfitting

In [ ]:

result_df = df_64[(df_64["WD"] == 0) | (df_64["WD"] == 0.01)]
result_df = result_df[(result_df["lr"] == 0.001)]
result_df = result_df.melt(id_vars=['epoch', 'WD'], value_vars=['train_loss', 'val_loss'],
                           var_name='Train Or Validation', value_name='loss')

In [ ]:
fig = go.Figure(data=px.line(result_df,x= "epoch",y="loss",color = "Train Or Validation",line_dash="WD"))

fig.update_layout(
    title='Train Loss vs Validation Loss for Learn Rate 0.001, Weight Decay 0.0, 0.01',
    xaxis_title='Epoch',
    yaxis_title='Cross Entropy Loss',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=24),
    yaxis_title_font=dict(size=24),
    xaxis_tickfont=dict(size=20),
    yaxis_tickfont=dict(size=20),
    legend_title_text='Train or Validation, Weight Decay',
    legend_title_font=dict(size=24),
    legend_font=dict(size=20)

)
fig.show()

From this we can actually see that the model does begin to overfit quite early on in training for either settings of the weight decay. The higher setting for weight decay does not seem to reduce overfitting at all for this model.
Something interesting is that the validation accuracy does also increase across the later epochs, even though validation loss is increasing too. It is difficult to find a reason for this as the CIFAR10 dataset is balanced.  